<a href="https://colab.research.google.com/github/solobala/pyda36/blob/main/Balakina_pyda36_DZ7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание к лекции «Понятие класса»

# Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса www.cbr-xml-daily.ru...ly_json.js

In [1]:
import requests

In [17]:
!wget -O "libs.zip" "https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/18487/libs.zip"  # как обычно качаем архив с нетологии

--2022-09-08 15:46:17--  https://u.netology.ngcdn.ru/backend/uploads/lms/attachments/files/data/18487/libs.zip
Resolving u.netology.ngcdn.ru (u.netology.ngcdn.ru)... 104.20.156.54, 104.20.157.54, 2606:4700:10::6814:9d36, ...
Connecting to u.netology.ngcdn.ru (u.netology.ngcdn.ru)|104.20.156.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2569 (2.5K) [application/x-zip-compressed]
Saving to: ‘libs.zip’

libs.zip            100%[===================>]   2.51K  --.-KB/s    in 0s      

2022-09-08 15:46:18 (11.4 MB/s) - ‘libs.zip’ saved [2569/2569]



In [18]:
from zipfile import ZipFile  # Подключаем библиотеку ZipFile из пакета zipfile

In [28]:
with ZipFile('/content/libs.zip', 'r') as zip_file: 
  zip_file.extract('libs/exchange.py', '.')

In [29]:
from pathlib import Path  # у нас нужный файл лежит в подкаталоге Libs
                          #  нужно в content перенести

for path in Path('/content/libs').glob('exchange.py'):
    path.replace(Path('/content') / path.name)

In [30]:
import sys
sys.path.append('/content/exchange.py')  # добавляем путь к библиотеке классов

In [33]:
from exchange import Rate

In [34]:
class RateExt(Rate):
    def __init__(self):
        super().__init__()
    # По рекомендации Башевого
    def name_by_max_rate(self): 
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        data_dict = r.json()['Valute']
        return( max(data_dict.items(), key=lambda x: x[1]['Value'])[1]['Name'])
    # def name_by_max_rate(self):
    #   """
    #   Возвращает название валюты (поле ‘Name’) 
    #   с максимальным значением курса
    #   """
    #   max_value = 0
    #   max_name = ''
    #   responce = self.exchange_rates()

    #   for name, param in responce.items():
    #       if param['Value'] > max_value:
    #           max_value = param['Value']
    #           max_name = param['Name']

    #   return max_name    

In [35]:
cc = RateExt()  # создаем экземпляр класса RateExt

In [36]:
print(cc.name_by_max_rate()) # применяем метод name_by_max_rate() для получения названия валюты с максимальным курсом

Китайских юаней


# Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [ ]:
class RateExtdiff(Rate):
    def __init__(self, format_='value', diff = False):
        super().__init__()
        self.diff = diff
        self.format = format_
    def make_format(self, currency, diff):
        """
        Возвращает информацию о валюте currency в следующих вариантах:
        - полная информация о валюте 
        при self.format = 'full':
        Rate('full').make_format('EUR', diff = False)
       
       {'CharCode': 'EUR',
        'ID': 'R01239',
        'Name': 'Евро',
        'Nominal': 1,
        'NumCode': '978',
        'Previous': 61.361,
        'Value': 61.1615}

        - информация только о курсе валюты 
        при self.format = 'value' и
        при self.diff = False:
        Rate('value').make_format('EUR', diff = False)
      
        61.1615

        - информация по изменению по сравнению с прошлым значением курса
         при self.format = 'value' и
        при self.diff = True:
        Rate('value', True).make_format('EUR', diff = True)
        
       -0.1995

        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']
            
            if self.format == 'value' and self.diff == True:
                return round(response[currency]['Value'] - response[currency]['Previous'],4)

        return 'Error'

    def eur(self, diff):
        """Возвращает курс евро на сегодня в формате self.format, если diff = False,
           и разницу в текущем и предыдущем значении курса, если diff = True
        """
        return self.make_format('EUR', diff)
    
    def usd(self,diff):
        """Возвращает курс доллара на сегодня в формате self.format, если diff = False,
           и разницу в текущем и предыдущем значении курса, если diff = True
        """
        return self.make_format('USD', diff)
    
    def brl(self,diff):
        """Возвращает курс бразильского реала на сегодня в формате self.forma, если diff = False,
           и разницу в текущем и предыдущем значении курса, если diff = True
        """
        return self.make_format('BRL', diff)

In [ ]:
r = RateExtdiff(format_ = 'value', diff = True)  # создаем экземпляр класса rate

In [ ]:
r.make_format( 'EUR', diff = True)

-0.1995

In [ ]:
r.eur(diff = True)

-0.1995

# Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия.

In [ ]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [ ]:
class Designer(Employee):
    def __init__(self, name, seniority, awards):
        super().__init__(name, seniority + awards)
        self.awards = 2

    def publish_grades(self):
        """Публикация атестаций и получения международных премий"""
        print(self.name,self.grades)

    def check_if_it_is_time_for_upgrade(self):
        # для каждой аттестацими увеличиваем счетчик на 1
        
        self.seniority += 1
        # для каждой международой премии увеличиваем счетчик на 2
        
        self.awards += 2
        
        # условие повышения сотрудника из презентации
        if (self.seniority + self.awards) % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [ ]:
alex = Designer('Александр', 0,2)

In [ ]:
for i in range(21):
    alex.check_if_it_is_time_for_upgrade()

Александр 2
Александр 2
Александр 2
Александр 2
Александр 2
Александр 2
Александр 2
Александр 3
Александр 3
Александр 3
Александр 3
Александр 3
Александр 3
Александр 3
Александр 4
Александр 4
Александр 4
Александр 4
Александр 4
Александр 4
Александр 4
